# Pagbuo ng AI Agents na may Persistent Memory gamit ang Cognee

Ang notebook na ito ay nagpapakita kung paano bumuo ng matatalinong AI agents na may advanced na kakayahan sa memorya gamit ang [**cognee**](https://www.cognee.ai/) - isang open source na AI memory na pinagsasama ang knowledge graphs, semantic search, at session management upang makalikha ng mga context-aware na AI systems.

## 🎯 Mga Layunin sa Pag-aaral

Sa pagtatapos ng tutorial na ito, mauunawaan mo kung paano:
- **Bumuo ng Knowledge Graphs na May Embeddings**: I-transform ang hindi organisadong teksto sa structured, queryable na kaalaman
- **Magpatupad ng Session Memory**: Lumikha ng multi-turn na mga pag-uusap na may awtomatikong pag-iingat ng konteksto
- **Mag-imbak ng Mga Pag-uusap**: Opsyonal na itago ang mahahalagang interaksyon sa pangmatagalang memorya para sa hinaharap na paggamit
- **Mag-query Gamit ang Natural na Wika**: I-access at gamitin ang historical na konteksto sa mga bagong pag-uusap
- **I-visualize ang Memorya**: Tuklasin ang mga relasyon sa knowledge graph ng iyong agent


## 🏗️ Ano ang Iyong Itatayo

Sa tutorial na ito, gagawa tayo ng isang **Coding Assistant** na may kakayahang magtago ng memorya na:

### 1. **Pagbuo ng Knowledge Base**
   - Kumukuha ng impormasyon tungkol sa profile at kakayahan ng developer
   - Nagpoproseso ng mga prinsipyo at pinakamahusay na kasanayan sa Python programming
   - Nagtatabi ng mga nakaraang pag-uusap sa pagitan ng mga developer at AI assistants

### 2. **Session-Aware na Mga Pag-uusap**
   - Pinapanatili ang konteksto sa maraming tanong sa parehong session
   - Awtomatikong iniimbak ang bawat pares ng tanong/sagot para sa mas mabilis na retrieval
   - Nagbibigay ng malinaw at may kontekstong sagot batay sa kasaysayan ng pag-uusap

### 3. **Pangmatagalang Memorya**
   - Nagtatabi ng mahahalagang pag-uusap sa pangmatagalang memorya
   - Kinukuha ang mga kaugnay na alaala mula sa knowledge base at mga nakaraang session upang magbigay ng impormasyon sa mga bagong interaksyon
   - Lumilikha ng lumalaking knowledge base na patuloy na bumubuti sa paglipas ng panahon

### 4. **Matalinong Retrieval ng Memorya**
   - Gumagamit ng graph-aware semantic search upang mahanap ang kaugnay na impormasyon sa lahat ng nakaimbak na kaalaman
   - Nagfi-filter ng mga paghahanap batay sa mga subgroup ng data (impormasyon ng developer vs. mga prinsipyo)
   - Pinagsasama ang maraming pinagmumulan ng data upang makapagbigay ng komprehensibong sagot


## 📋 Mga Paunang Kailangan at Setup

### Mga Kinakailangan sa Sistema

Bago magsimula, tiyakin na mayroon ka ng mga sumusunod:

1. **Python Environment**
   - Python 3.9 o mas mataas
   - Virtual environment (inirerekomenda)
   
2. **Redis Cache** (Kinakailangan para sa Session Management)
   - Lokal na Redis: `docker run -d -p 6379:6379 redis`
   - O gumamit ng isang managed Redis service
   
3. **LLM API Access**
   - OpenAI API key o iba pang provider (tingnan ang [dokumentasyon](https://docs.cognee.ai/setup-configuration/llm-providers))

4. **Database Configuration**
   - Walang kinakailangang configuration bilang default. Gumagamit ang Cognee ng file-based databases (LanceDB at Kuzu)
   - Opsyonal, maaari kang mag-setup ng Azure AI Search bilang vector store (tingnan ang [dokumentasyon](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch))

### Configuration ng Environment

Gumawa ng `.env` file sa iyong project directory na may mga sumusunod na variable:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Pag-unawa sa Arkitektura ng Memorya ng Cognee

### Paano Gumagana ang Cognee

Ang Cognee ay nagbibigay ng isang sopistikadong sistema ng memorya na higit pa sa simpleng key-value storage:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### Mga Pangunahing Komponent:

1. **Knowledge Graph**: Nagtatago ng mga entidad, relasyon, at semantikong koneksyon
2. **Vector Embeddings**: Nagpapahintulot ng semantikong paghahanap sa lahat ng nakaimbak na impormasyon
3. **Session Cache**: Pinapanatili ang konteksto ng pag-uusap sa loob at labas ng mga sesyon
4. **NodeSets**: Inaayos ang data sa mga lohikal na kategorya para sa mas tiyak na retrieval

### Mga Uri ng Memorya sa Tutorial na Ito:

- **Persistent Memory**: Pangmatagalang imbakan sa knowledge graph
- **Session Memory**: Pansamantalang konteksto ng pag-uusap sa Redis cache
- **Semantic Memory**: Batay sa vector na paghahanap ng pagkakatulad sa lahat ng data


## 📦 Mag-install ng Mga Kinakailangang Package

I-install ang Cognee na may suporta sa Redis para sa session management:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 I-set up ang Kapaligiran at I-load ang mga Library

Siguraduhin na:
1. Tumatakbo ang Redis (hal., gamit ang Docker: `docker run -d -p 6379:6379 redis`)
2. Naka-set ang mga environment variable bago i-import ang mga cache module
3. Kung kinakailangan, i-restart ang kernel at patakbuhin ang mga cell nang sunud-sunod

Ang sumusunod na cell ay gagawin ang mga sumusunod:
1. I-load ang mga environment variable mula sa `.env`
2. I-configure ang Cognee gamit ang iyong LLM settings
3. I-enable ang caching para sa session management
4. Siguraduhing maayos na nakakonekta ang lahat ng mga bahagi


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 I-configure ang mga Direktoryo ng Storage

Gumagamit ang Cognee ng dalawang magkahiwalay na direktoryo para sa mga operasyon nito:
- **Data Root**: Nagtatabi ng mga dokumentong na-ingest at mga naprosesong data
- **System Root**: Naglalaman ng knowledge graph database at metadata ng sistema

Gagawa tayo ng magkakahiwalay na direktoryo para sa tutorial na ito tulad ng sumusunod:


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 I-reset ang Estado ng Memorya

Bago tayo magsimula sa paggawa ng ating sistema ng memorya, tiyakin muna natin na nagsisimula tayo nang malinis.

> 💡 **Tip**: Maaari mong laktawan ang hakbang na ito kung nais mong panatilihin ang mga umiiral na alaala mula sa iyong mga nakaraang pagtakbo kapag ginamit mo ang notebook na ito sa hinaharap.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 Bahagi 1: Pagbuo ng Knowledge Base

### Mga Pinagmulan ng Data para sa Ating Developer Assistant

Tatlong uri ng data ang ating gagamitin upang makabuo ng komprehensibong knowledge base:

1. **Profile ng Developer**: Personal na kadalubhasaan at teknikal na background
2. **Mga Pinakamahusay na Praktis sa Python**: Ang Zen of Python na may praktikal na gabay
3. **Mga Makasaysayang Usapan**: Mga nakaraang Q&A session sa pagitan ng mga developer at AI assistants

Ang magkakaibang data na ito ay nagbibigay-daan sa ating agent na:
- Maunawaan ang teknikal na konteksto ng user
- Mag-apply ng pinakamahusay na praktis sa mga rekomendasyon
- Matuto mula sa mga nakaraang matagumpay na interaksyon


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 Proseso ng Datos patungo sa Knowledge Graph

Ngayon, babaguhin natin ang ating raw text sa isang nakaayos na memorya. Ang prosesong ito:

1. **Nagdaragdag ng datos sa NodeSets**: Inaayos ang impormasyon sa lohikal na mga kategorya
   - `developer_data`: Profile ng developer at mga pag-uusap
   - `principles_data`: Mga pinakamahusay na kasanayan at gabay sa Python

2. **Pinapatakbo ang Cognify Pipeline**: Kinukuha ang mga entity, relasyon, at gumagawa ng embeddings
   - Tinutukoy ang mga pangunahing konsepto
   - Lumilikha ng semantikong koneksyon sa pagitan ng magkakaugnay na impormasyon
   - Gumagawa ng vector embeddings

Maaaring tumagal ito ng ilang sandali habang pinoproseso ng LLM ang teksto at binubuo ang istruktura ng graph:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 I-visualize ang Knowledge Graph

Tuklasin natin ang istruktura ng ating knowledge graph. Ang visualization ay nagpapakita ng:
- **Nodes**: Mga entity na nakuha mula sa teksto (mga konsepto, teknolohiya, tao)
- **Edges**: Mga relasyon at koneksyon sa pagitan ng mga entity
- **Clusters**: Mga kaugnay na konsepto na pinagsama batay sa semantikong pagkakatulad

Buksan ang nalikhang HTML file sa iyong browser upang interaktibong tuklasin ang graph:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 Pagyamanin ang Memorya gamit ang Memify

Ang `memify()` na function ay sinusuri ang knowledge graph at bumubuo ng matatalinong alituntunin tungkol sa datos. Ang prosesong ito:
- Tinutukoy ang mga pattern at pinakamahusay na mga kasanayan
- Lumilikha ng mga praktikal na gabay batay sa nilalaman
- Nagpapalaganap ng ugnayan sa pagitan ng iba't ibang larangan ng kaalaman

Ang mga alituntuning ito ay tumutulong sa ahente na gumawa ng mas maalam na desisyon kapag sumasagot ng mga tanong. Ang pagkuha ng pangalawang biswal na representasyon ay tumutulong sa iyo na maikumpara kung paano nagiging mas siksik ang graph kapag napagyaman.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 Bahagi 2: Matalinong Pagkuha ng Impormasyon

### Demonstrasyon 1: Pagsasama ng Kaalaman mula sa Iba't ibang Dokumento

Ngayon na nabuo na ang ating knowledge graph, subukan natin kung paano pinagsasama ng Cognee ang impormasyon mula sa iba't ibang mapagkukunan upang sagutin ang masalimuot na mga tanong.

Ang unang query ay nagpapakita ng:
- **Semantic na pag-unawa**: Paghahanap ng mga kaugnay na konsepto kahit hindi ito tahasang binanggit
- **Cross-referencing**: Pagsasama ng profile ng developer sa mga prinsipyo ng Python
- **Contextual na pangangatwiran**: Paglalapat ng pinakamahusay na mga kasanayan sa mga tiyak na implementasyon

### Demonstrasyon 2: Piniling Paghahanap gamit ang NodeSets

Ang pangalawang query ay nagpapakita kung paano mag-target ng mga tiyak na subset ng knowledge graph:
- Gumagamit ng `node_name` parameter upang maghanap lamang sa loob ng `principles_data`
- Nagbibigay ng nakatuon na sagot mula sa isang tiyak na domain ng kaalaman
- Kapaki-pakinabang kapag kailangan mo ng impormasyon na tiyak sa isang domain


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 Bahagi 3: Pag-set up ng Pamamahala ng Session

### Pag-enable ng Memorya ng Usapan

Ang pamamahala ng session ay mahalaga para mapanatili ang konteksto sa maraming interaksyon. Dito natin gagawin ang sumusunod:

1. **I-initialize ang Konteksto ng User**: Gumawa o kunin ang profile ng user para sa pagsubaybay ng session
2. **I-configure ang Cache Engine**: Kumonekta sa Redis para sa pag-iimbak ng kasaysayan ng usapan
3. **I-enable ang Mga Variable ng Session**: Mag-set up ng mga variable ng konteksto na nananatili sa bawat query

> ⚠️ **Mahalaga**: Kinakailangan na tumatakbo ang Redis at `CACHING=true` sa iyong environment


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ Helper Function: Tingnan ang Kasaysayan ng Session

Ang utility function na ito ay nagbibigay-daan sa atin na suriin ang kasaysayan ng pag-uusap na nakaimbak sa Redis. Ito ay kapaki-pakinabang para sa:
- Pag-debug ng pamamahala ng session
- Pag-verify na ang mga pag-uusap ay nai-cache
- Pag-unawa kung anong konteksto ang magagamit para sa ahente


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## Session 1: Async Support Lab — Unang Tanong

Simulan ang sesyon ng `async-support-lab` sa pamamagitan ng pagtatanong tungkol sa mga telemetry-friendly na asyncio patterns para sa isang napakalaking web scraper. Alam na ng graph ang tungkol sa asyncio, aiohttp, at mga monitoring practices, kaya ang sagot ay dapat sumasalamin sa mga naunang pag-uusap habang iniangkop ang sagot sa bagong tanong.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## Suriin ang Memorya ng Session 1 Pagkatapos ng Unang Palitan

Ang pagtakbo ng `show_history(session_1)` kaagad pagkatapos ng unang tanong ay nagkukumpirma na isinulat ng Cognee ang parehong prompt at completion sa Redis. Makikita mo ang isang entry na may gabay sa concurrency.


In [ ]:
await show_history(session_1)

## Session 1: Pagsusuri sa Data Models

Susunod nating tanungin, "Kailan ko dapat piliin ang dataclasses kumpara sa Pydantic?" gamit ang parehong session id. Dapat pagsamahin ni Cognee ang mga prinsipyo ng Python kasama ang mga naunang pag-uusap tungkol sa FastAPI upang magbigay ng mas detalyado at masusing payo—ipinapakita na ang konteksto ay naipapasa sa loob ng isang pinangalanang session.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## Kumpirmahin na ang Kasaysayan ng Session 1 ay Naglalaman ng Parehong Mga Palitan

Ang isa pang `show_history(session_1)` na tawag ay dapat magpakita ng dalawang Q&A na entry. Ito ay tumutugma sa "memory replay" na hakbang ng Mem0 lab at nagpapatunay na ang mga karagdagang palitan ay nagdaragdag sa parehong transcript.


In [ ]:
await show_history(session_1)

## Session 2: Thread ng Pagsusuri ng Disenyo — Bagong Session

Para ipakita ang paghihiwalay sa pagitan ng mga thread, pinapatakbo namin ang `design-review-session` at humihingi ng gabay sa pag-log para sa pagsusuri ng insidente. Kahit pareho ang base ng kaalaman, ang bagong session id ay nagpapanatili ng hiwalay na mga transcript.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## Pagsusuri ng Session 2 Kasaysayan

`show_history(session_2)` ay dapat maglista lamang ng pares ng prompt/response para sa design-review. Ihambing ito sa Session 1 upang ipakita kung paano pinapanatili ng Cognee ang mga independiyenteng transcript habang muling ginagamit ang ibinahaging knowledge graph.


In [ ]:
await show_history(session_2)

## Buod

Binabati kita! Nabigyan mo na ang iyong coding assistant ng isang tunay na pangmatagalang memory layer na pinapagana ng Cognee.

Sa tutorial na ito, kinuha mo ang raw na nilalaman ng developer (code, dokumento, chat) at ginawang isang graph + vector memory na maaaring hanapin, pag-isipan, at patuloy na pagbutihin ng iyong agent.

Ano ang Iyong Natutunan

1. **Mula sa raw na teksto patungo sa AI memory**: Paano ini-ingest ng Cognee ang hindi istrukturang data at ginagawang matalino, mahanap na memory gamit ang pinagsamang vector + knowledge graph na arkitektura.

2. **Pagpapayaman ng graph gamit ang memify**: Paano lampasan ang simpleng paggawa ng graph at gamitin ang memify upang magdagdag ng mga derived facts at mas mayamang relasyon sa ibabaw ng iyong umiiral na graph.

3. **Iba't ibang estratehiya sa paghahanap**: Paano mag-query ng memory gamit ang iba't ibang uri ng paghahanap (graph-aware Q&A, RAG-style completion, insights, raw chunks, code search, atbp.) depende sa pangangailangan ng iyong agent.

4. **Visual na eksplorasyon**: Paano inspeksyunin at i-debug ang ginawa ng Cognee gamit ang mga graph visualization at ang Cognee UI, upang makita mo kung paano nakaayos ang kaalaman.

5. **Session-aware memory**: Paano pagsamahin ang per-session context sa persistent semantic memory upang maalala ng mga agent ang mga impormasyon sa bawat run nang hindi nagkakaroon ng pagtagas ng impormasyon sa pagitan ng mga user.


## Mga Pangunahing Punto
1. Memorya bilang isang Knowledge Graph na sinusuportahan ng Embeddings

    - **Naka-istrukturang pag-unawa**: Pinagsasama ng Cognee ang isang vector store at isang graph store upang ang iyong data ay parehong mahanap batay sa kahulugan at konektado sa pamamagitan ng mga relasyon. Gumagamit ang Cognee ng mga file-based na database bilang default (LanceDB para sa vector-, Kuzu para sa graph database).

    - **Pagkuha na may kamalayan sa relasyon**: Ang mga sagot ay maaaring batay hindi lamang sa “katulad na teksto,” kundi pati na rin sa kung paano nauugnay ang mga entity.

    - **Buhay na memorya**: Ang memory layer ay umuunlad, lumalago, at nananatiling mahanap bilang isang konektadong graph.

2. Mga Mode ng Paghahanap at Pangangatwiran
    - **Hybrid retrieval**: Ang paghahanap ay pinagsasama ang vector similarity, graph structure, at LLM reasoning, mula sa raw chunk lookup hanggang sa graph-aware na pagsagot sa tanong.

    - **Iangkop ang mode sa trabaho**: Gumamit ng completion-style modes kapag gusto mo ng natural na sagot sa wika, at chunk/summary/graph modes kapag kailangan ng iyong agent ng raw na konteksto o upang magmaneho ng sariling pangangatwiran.

3. Personalized, Session-Aware Agents
    - **Session context + long-term memory**: Pinapanatili ng Cognee ang maikling “thread” context na hiwalay mula sa pangmatagalang memorya sa antas ng user o organisasyon.

## Mga Aplikasyon sa Tunay na Mundo

1. **Vertical AI Agents**

    Gamitin ang pattern mula sa notebook na ito upang paganahin ang domain-smart copilots na nakabase sa Cognee bilang retrieval at reasoning core:

- **Developer copilots**: Code review, pagsusuri ng insidente, at mga assistant sa arkitektura na naglalakbay sa code, APIs, design docs, at mga ticket bilang isang memory graph.

- **Customer-facing copilots**: Mga support o success agents na kumukuha mula sa product docs, FAQs, CRM notes, at mga nakaraang ticket gamit ang graph-aware retrieval at mga sagot na may citation.

- **Internal expert copilots**: Mga assistant sa polisiya, legal, o seguridad na nangangatuwiran sa interconnected na mga patakaran, gabay, at mga desisyon sa kasaysayan sa halip na mga hiwalay na PDF.

    Ang Cognee ay tahasang nakaposisyon bilang persistent, accurate memory para sa AI agents, na nagbibigay ng isang buhay na knowledge graph na maaaring gamitin sa likod ng iyong agent at pumalit sa ad-hoc na kombinasyon ng vector stores at custom graph code.

2. **Pag-iisa ng Data Silos sa Isang Memorya**

    Ang parehong approach ay tumutulong din sa pagbuo ng isang unified memory layer sa mga magkakahiwalay na sources:

- **Mula sa silos patungo sa isang graph**: I-ingest ang structured (hal., databases) at unstructured data (hal., docs, chats) sa isang graph na sinusuportahan ng embeddings, sa halip na magkakahiwalay na indices para sa bawat sistema.

- **Pangangatwiran sa cross-source na may citations**: Magpatakbo ng multi-step reasoning sa lahat—“i-join” ang logs, metrics, at docs sa pamamagitan ng graph—at magbalik pa rin ng grounded na sagot na may provenance.

- **Knowledge hubs**: Para sa mga domain tulad ng banking o edukasyon, ginagamit na ang Cognee upang pag-isahin ang PDFs, internal systems, at app data sa isang knowledge graph na may vectors upang ang mga agents ay makasagot ng mga tanong na may eksaktong, cited na konteksto.

## Mga Susunod na Hakbang

Na-implement mo na ang core memory loop. Narito ang mga natural na extension na maaari mong subukan sa iyong sarili (tingnan ang [Cognee documentation](https://docs.cognee.ai/) para sa mga detalye):

1. **Mag-eksperimento sa temporal awareness**: I-on ang temporal cognify upang mag-extract ng mga event at timestamps mula sa teksto.

2. **Magpakilala ng ontology-driven reasoning**: Tukuyin ang isang OWL ontology para sa iyong domain. Gamitin ang suporta ng Cognee sa ontology upang ang mga na-extract na entity at relasyon ay nakabase sa schema na iyon, na nagpapabuti sa kalidad ng graph at mga sagot na partikular sa domain.

3. **Magdagdag ng feedback loop**: Hayaan ang Cognee na ayusin ang graph edge weights mula sa aktwal na feedback ng user, upang ang retrieval ay patuloy na bumuti sa paglipas ng panahon sa halip na manatiling static.

4. **I-tune para sa personalization at session behavior**: Gamitin ang user IDs, tenants, at datasets upang bigyan ang bawat tao o team ng kanilang sariling view sa shared memory engine.

5. **Mag-scale out sa mas kumplikadong agents**: I-plug ang Cognee sa agent frameworks upang bumuo ng multi-agent systems na lahat ay nagbabahagi ng parehong memory layer. *Microsoft Agent Framework x Cognee plugin ay paparating na.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, mangyaring tandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na awtoritatibong pinagmulan. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na dulot ng paggamit ng pagsasaling ito.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
